In [12]:
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from scipy.interpolate import interp1d
import torch.nn.init as init

## Custom Function Activation -- Module

In [13]:
class CustomFunction(nn.Module):
    def __init__(self,function):
        super().__init__()
        self.function = function
        
    def forward(self, x):
        global lim_inf,lim_sup
        x = torch.clamp(x, lim_inf, lim_sup)
        x = x.detach().numpy()
        return torch.tensor(self.function(x), dtype=torch.float32)

## Class MLP_CUSTOM

In [14]:
class MLP_Custom(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, genome, pars):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.custom_activation = CustomFunction(self.act_func_generator(genome))
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        if pars is not None:
            self.set_weights(pars)
        else:
            self.init_weights()
        
        global seed
        if seed is not None:
            torch.manual_seed(seed)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.custom_activation(x)
        x = self.fc2(x)
        return x
    
    def act_func_generator(self, genome):
        global lim_inf, lim_sup
        x_dom = np.linspace(lim_inf, lim_sup, len(genome))
        function = interp1d(x_dom, genome, kind='cubic', fill_value="extrapolate")
        return function
    
    def init_weights(self):
        init.xavier_uniform_(self.fc1.weight)
        init.zeros_(self.fc1.bias)
        init.xavier_uniform_(self.fc2.weight)
        init.zeros_(self.fc2.bias)
    
    def set_weights(self, pars):
        with torch.no_grad():
            for param, p in zip(self.parameters(), pars):
                param.copy_(torch.tensor(p))



In [15]:
 def print_par(index):
  for name, param in net_pop[index].network.named_parameters():
   print(f'Ind:{index},{name}: {np.round(np.array(param.data),3).reshape(1, -1)}')

In [24]:
class Individual(object):
    def __init__(self,genome,input_size,hidden_size,output_size,pars=None):
        self.genome  = genome
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.score= 0      
        self.network = MLP_Custom(self.input_size,self.hidden_size,self.output_size,self.genome,pars)
        self.init_pars=self.network.parameters()
        self.trained_pars= None
    
    # Create chromosome
    @classmethod
    def create_chromosome(self,level_inf,level_sup,step,dna_length,inter):  
     if inter == False:
      dna = np.random.choice(np.arange(level_inf, level_sup, step), size=dna_length, replace=True)
     else:
      dna = np.random.randint(level_inf, level_sup, dna_length)
     return dna
    
    def training(self,lr,epochs,X,Y):
     mse_loss = nn.MSELoss()
     optimizer = optim.Adam(self.network.parameters(), lr)
    
     loss_values_custom = []
     for epoch in range(epochs):
        self.network.train()
        optimizer.zero_grad()
        output = self.network(X)
        loss = mse_loss(output, Y)
        #print(loss.item())
        loss_values_custom.append(loss.item())
        loss.backward()
        optimizer.step()
     self.score = loss_values_custom[-1] 
     self.trained_pars=self.network.parameters()
     
    @classmethod
    def selection(self,net_pop,k):
      subset = random.sample(net_pop, k)
      parent = min(subset, key=lambda x: x.score) 
      return parent
    
    @staticmethod
    def mutation(genome,method='Flip Bit Mutation',hm_flip=3,hm_swap=1):
        global level_inf,level_sup,step
        if method == 'Flip Bit Mutation':
         print(f'Original genome:{genome}')
         for i in random.sample(range(len(genome)), hm_flip):
          genome[i] = round(np.random.choice(np.arange(level_inf, level_sup, step), size=1, replace=True)[0],2)
         print(f'Mutated genome:{genome}')         
         return genome
        else:
         for _ in range(hm_swap):
          i, j = random.sample(range(len(genome)), 2)
          genome[i], genome[j] = genome[j], genome[i]
          return genome
    
    # Crossover function
    def crossover(self,second_parent,id,point=2,mut=False):
        child_chromosome=[]
        if mut is False:
            if id == 1:
             for x,y in zip(self.genome,second_parent.genome):
              child_chromosome.append(round(random.choice([x, y]),2))
             return  Individual(child_chromosome,self.input_size,self.hidden_size,self.output_size)
            elif id == 2:
             child_chromosome= self.genome[:point]+second_parent.genome[point:]
             return  Individual(child_chromosome,self.input_size,self.hidden_size,self.output_size)
        else:
             if id == 1:
              for x,y in zip(self.genome,second_parent.genome):
               child_chromosome.append(round(random.choice([x, y]),2))
              child_chromosome=Individual.mutation(child_chromosome)
              return  Individual(child_chromosome,self.input_size,self.hidden_size,self.output_size)
             elif id == 2:
              child_chromosome= self.genome[:point]+second_parent.genome[point:]
              return  Individual(child_chromosome,self.input_size,self.hidden_size,self.output_size)

In [29]:
# ----------------Activation Parameters-------------------
# Codomain limits
level_inf=0
level_sup=3
inter = False
step=0.01
# Domain limits
lim_inf=-3
lim_sup=3
# DNA Length
dna_length=10
# ----------------Training Parameters-------------------
# input_size
input_size=1
# hidden_size
hidden_size=10
# output_size
output_size=1
# Epochs
epochs=1000
# Learning Rate
lr=0.01
# Seed parameters generation
seed = 42
# ----------------Genetic Algorithm Parameters-------------------
# Number of networks
pop_size = 5
# Number of generations
N_gens=2
# Selection subject number
k=5
# Crossover and Mutation percentage
mut_per= int((pop_size*30)/100)
# Only Crossover percentage
cro_per =pop_size-mut_per
## ----------------Initilization----------------------------------
net_pop=[]
for i in range(pop_size):
    # Generation DNA for activation function
    genome = Individual.create_chromosome(level_inf,level_sup,step,dna_length,inter)
    # Networks Initialization
    net_pop.append(Individual(genome,input_size,hidden_size,output_size))

In [26]:
 print_par(0)

Ind:0,fc1.weight: [[-0.627  0.57   0.123 -0.24   0.456  0.115  0.597  0.081 -0.233  0.198]]
Ind:0,fc1.bias: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ind:0,fc2.weight: [[-0.2    0.311  0.659  0.427 -0.323  0.426  0.132  0.375 -0.45  -0.731]]
Ind:0,fc2.bias: [[0.]]


In [27]:
X=np.linspace(-2,2,1000).reshape(-1,1)
Y=np.sin(X)
X_tensor = torch.tensor(X,dtype=torch.float32)
Y_tensor = torch.tensor(Y,dtype=torch.float32)

In [28]:
for i in range(N_gens):
    new_pop=[]
    
    # Training
    print(f'Training generation {i} started')
    for j in range(pop_size):
     net_pop[j].training(lr,epochs,X_tensor,Y_tensor)
    print(f'Training generation {i} endend')
     
    # Selection and Crossover
    print(f'Starting selection and cross-over phase')
    for p in range(cro_per):
     first_parent=Individual.selection(net_pop,k)
     second_parent=Individual.selection(net_pop,k)
     child = first_parent.crossover(second_parent,1,2)
     print(f'Ge gen1:{first_parent.genome}')
     print(f'Ge gen2:{second_parent.genome}')
     print(f'Child {p}:{child.genome}')
     new_pop.append(child)
    print(f'Starting mutation phase')
    # Mutuation
    for p in range(mut_per):
     first_parent=Individual.selection(net_pop,k)
     second_parent=Individual.selection(net_pop,k)
     child = first_parent.crossover(second_parent,1,2,mut=True)
     new_pop.append(child)
     print(f'Ge gen1:{first_parent.genome}')
     print(f'Ge gen2:{second_parent.genome}')
     print(f'Child {p}:{child.genome}')   
    net_pop=new_pop


Training generation 0 started
Training generation 0 endend
Starting selection and cross-over phase
Ge gen1:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Ge gen2:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Child 0:[1.32, 1.8, 0.94, 0.75, 1.01, 0.21, 2.81, 1.49, 1.58, 1.84]
Ge gen1:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Ge gen2:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Child 1:[1.32, 1.8, 0.94, 0.75, 1.01, 0.21, 2.81, 1.49, 1.58, 1.84]
Ge gen1:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Ge gen2:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Child 2:[1.32, 1.8, 0.94, 0.75, 1.01, 0.21, 2.81, 1.49, 1.58, 1.84]
Ge gen1:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Ge gen2:[1.32 1.8  0.94 0.75 1.01 0.21 2.81 1.49 1.58 1.84]
Child 3:[1.32, 1.8, 0.94, 0.75, 1.01, 0.21, 2.81, 1.49, 1.58, 1.84]
Starting mutation phase
Original genome:[1.32, 1.8, 0.94, 0.75, 1.01, 0.21, 2.81, 1.49, 1.58, 1.84]
Mutated genome:[1.32, 1.8, 0.94, 0.75, 1.01, 1.51

In [22]:
 print_par(2)

Ind:2,fc1.weight: [[-0.627  0.57   0.123 -0.24   0.456  0.115  0.597  0.081 -0.233  0.198]]
Ind:2,fc1.bias: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Ind:2,fc2.weight: [[-0.2    0.311  0.659  0.427 -0.323  0.426  0.132  0.375 -0.45  -0.731]]
Ind:2,fc2.bias: [[0.]]
